# The effect of COVID-19 on deliveredKwh
We designed three machine learning-based model (LSTM-based) that to predict the value of "dailyDeleivered". 

We monitored the prediction error of the pre and post COVID data to observe the effect of COVID-19 on the prediction error. 
We also observed the sign of the prediction error to see whether the predicted dailyDelivered values are greater or less than the actula values.
The following code and results are for a dataset that contains data of residential premises.

The prediction error for post-covid data is greater than the one for pre-covid data.

In [498]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from matplotlib.pyplot import locator_params
from numpy import genfromtxt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize
from operator import add
import tensorflow as tf
#rom keras.backend import tensorflow_backend as K
import statsmodels
import statistics
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import date
from dateutil import parser

In [499]:
def temporalize(arr,win_size,step_size,features,add_index=0):
  """
  arr: any 2D array whose columns are distinct variables and 
    rows are data records at some timestamp t
  win_size: size of data window (given in data points)
  step_size: size of window step (given in data point)
  
  Note that step_size is related to window overlap (overlap = win_size - step_size), in 
  case you think in overlaps."""
  #
  dataFrameTimeseries=pd.DataFrame()
  #
  w_list = list()
  n_records = arr.shape[0]
  remainder = (n_records - win_size) % step_size 
  num_windows = 1 + int((n_records - win_size - remainder) / step_size)
  for k in range(num_windows):
    w_list.append(arr[k*step_size:win_size-1+k*step_size+1])
    #
    #convert the matrix to data frame
    dataFrameTemp=pd.DataFrame(data=arr[k*step_size:win_size-1+k*step_size+1], columns=features)
    dataFrameTemp["timeseriesId"]=k+add_index
    dataFrameTimeseries=pd.concat([dataFrameTimeseries,dataFrameTemp])
    #
  return np.array(w_list),dataFrameTimeseries


In [500]:
def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            pass
        
        try:
            import unicodedata
            unicodedata.numeric(s)
            return True
        except (TypeError, ValueError):
            pass
        return False 

In [501]:
def changeDates(dateList):
    daysList = []
    firstDate = parser.parse(dateList[0])
    for date in dateList:
        dateObject = parser.parse(date) - firstDate
        #daysList.append(dateObject.timetuple().tm_yday)
        daysList.append(dateObject.days)
    
    return daysList

In [502]:
def plot_(modelName,h_axis,v1_axis,v2_axis, title="Daily Delivered (kWh)"):
    h_axis = changeDates(h_axis)
    plt.rcParams.update({'font.size': 15})
    plt.tight_layout()
    plt.figure(figsize=(30,6))
    plt.plot(h_axis, v2_axis, label="predictedLoad")
    plt.plot(h_axis, v1_axis, label="dailyDelivered")
    plt.ylim(top = max(max(v1_axis), max(v2_axis)) + 0.3)
    plt.legend(loc="upper right")
    plt.title(title)
    plt.savefig(modelName+".png", facecolor='white', transparent=False)

In [503]:
# split into train and test sets
def split_(dataset, train_size=0.67, dimension=2):
    #train_size = int(len(dataset) * train_size)
    train_size = 365
    test_size = len(dataset) - train_size
    train=[]
    test=[]
    if dimension==1:
        train, test = dataset[0:train_size], dataset[train_size:len(dataset)]
    else:
        train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    return train,test

In [504]:
def lstmAutoencoer(modelName,X,n_features,dailyDeliveredIndex=3,trainig_percentage=0.76):
    #LSTM-Autoencoder
    model = Sequential()
    model.add(LSTM(20,activation='tanh', input_shape=(win_size,n_features-2), return_sequences=False))
    model.add(RepeatVector(win_size))
    model.add(LSTM(20, activation='tanh', return_sequences=True))
    #2: excludes id and time from analysis
    model.add(TimeDistributed(Dense(n_features-2)))
    model.compile(optimizer='adam', loss="mse")
    model.summary()

    #remove id and time for trainig
    X_preprocessed=np.asarray(np.delete(X,[0,1],axis=2)).astype(np.float32)
    #train on precovid data and test on all dataset
    train,test=split_(X_preprocessed,trainig_percentage)

    model.fit(train, train, epochs=500, verbose=1)
    #prediction error on pre-covid data (training data)
    yhat1 = model.predict(train, verbose=0)
    prediction_error1=np.square(train-yhat1).mean(axis=None).ravel()
    print("prediction error on pre-covid data")
    print("**********************************")
    print(prediction_error1)

    #prediction error on post-covid data (testing data)
    yhat2 = model.predict(test, verbose=0)
    prediction_error2=np.square(test-yhat2).mean(axis=None).ravel()
    print("prediction error on post-covid data")
    print("**********************************")
    print(prediction_error2)

    print("Error sign (actual-prediction) for post covid data: If Positive--> actual is greater than predicted")
    print("**********************************")
    print((test-yhat2).mean(axis=None))

    #predict the entire data
    yhat = model.predict(X_preprocessed, verbose=1)
    #this structure is because overlap is win_size-1. Uncomment the commented lines for overlap=0
    #h_axis=np.array(dataFrameTimeseries["time"])
    h_axis=np.array(X)[:,0,1].flatten()
    #v1_axis=np.array(dataFrameTimeseries["residential-dailyDelivered"])
    v1_axis=np.array(X)[:,0,dailyDeliveredIndex].flatten()
    #v2_axis=np.array(yhat)[:,:,1].flatten()
    v2_axis=np.array(yhat)[:,0,1].flatten()
    plot_(modelName,h_axis,v1_axis,v2_axis)

In [505]:
#Original LSTM
def lstm_(modelName,X,Y,n_features,excludedIndexes=[0,1],trainig_percentage=0.76):
    model = Sequential()
    model.add(LSTM(100, activation=('tanh'), dropout=0.2, recurrent_dropout=0.2, stateful = False, input_shape = (win_size,n_features-len(excludedIndexes))))
    model.add(Dense(1))
    model.compile(loss=keras.losses.mean_squared_error, optimizer='rmsprop')
    model.summary()

    #Exclude indexes for analysis in training
    X_preprocessed=np.asarray(np.delete(X,excludedIndexes,axis=2)).astype(np.float32)
    #train on precovid data and test on all dataset
    Xtrain,Xtest=split_(X_preprocessed,trainig_percentage)
    Ytrain,Ytest=split_(Y,trainig_percentage,1)

    model.fit(Xtrain, Ytrain,epochs=100,verbose=1)
    #prediction error on pre-covid data (training data)
    yhat1 = model.predict(Xtrain, verbose=0)
    prediction_error1=np.square(Ytrain-yhat1).mean(axis=None).ravel()
    print("prediction error on pre-covid data")
    print("**********************************")
    print(prediction_error1)

    #prediction error on post-covid data (testing data)
    yhat2 = model.predict(Xtest, verbose=0)
    prediction_error2=np.square(Ytest-yhat2).mean(axis=None).ravel()
    print("prediction error on post-covid data")
    print("**********************************")
    print(prediction_error2)

    print("Error sign (actual-prediction) for post covid data: If Positive--> actual is greater than predicted")
    print("**********************************")
    print((Ytest-yhat2).mean(axis=None))

    #predict the entire data
    yhat=model.predict(X_preprocessed)

    h_axis=dataset["time"]
    v1_axis=dataset["dailyDelivered"]
    v2_axis=yhat.flatten()
    temp=np.zeros(win_size)
    v2_axis=np.concatenate([temp,v2_axis])
    plot_(modelName,h_axis,v1_axis,v2_axis)

In [506]:
#set window sizes and their overlap
win_size=10
overlap=win_size-1

#read data
dataset = pd.read_csv('../ProcessedData/processed_GS_Energy.csv', engine='python')

#preprocess data
dataset=dataset.fillna(-1)
for column in dataset.columns:
    if is_number(dataset.iloc[1][column]) and column!="id":
        min_max=MinMaxScaler(feature_range=(0, 1))
        dataset[[column]]=min_max.fit_transform(dataset[[column]])

#set variables based on the data attributes
n_features=len(dataset.columns.values)
dailyDeliveredIndex=np.where(dataset.columns.values=="dailyDelivered")
dailyDeliveredIndex=dailyDeliveredIndex[0][0]
X,dataFrameTimeseries=temporalize(dataset.to_numpy(),win_size,win_size-overlap,dataset.columns.values)

In [507]:
#model1:
print("**********Model1: Original LSTM*******************************")
#remove id (index=0), time (index=1), and previous value of dailyDelivered for trainig
excludedIndexes=[0,1,dailyDeliveredIndex]
lstm_("Model1",X[0:-1,:],dataset.to_numpy()[win_size:,dailyDeliveredIndex].astype(np.float32),n_features,excludedIndexes,trainig_percentage=0.76)

#model2:
print("**********Model2: Original LSTM*******************************")
#remove id and time for trainig
excludedIndexes=[0,1]
lstm_("Model2",X[0:-1,:],dataset.to_numpy()[win_size:,dailyDeliveredIndex].astype(np.float32),n_features,excludedIndexes,trainig_percentage=0.76)

#model3:
print("**********Model3: LSTM-Autoencoder*******************************")
lstmAutoencoer("Model3",X,n_features,dailyDeliveredIndex,trainig_percentage=0.76)

**********Model1: Original LSTM*******************************
Model: "sequential_92"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_94 (LSTM)               (None, 100)               41200     
_________________________________________________________________
dense_92 (Dense)             (None, 1)                 101       
Total params: 41,301
Trainable params: 41,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
12/12 [==============================] - 2s 15ms/step - loss: 0.1142
Epoch 2/100
12/12 [==============================] - 0s 13ms/step - loss: 0.0366
Epoch 3/100
12/12 [==============================] - 0s 11ms/step - loss: 0.0309
Epoch 4/100
12/12 [==============================] - 0s 12ms/step - loss: 0.0306
Epoch 5/100
12/12 [==============================] - 0s 12ms/step - loss: 0.0285
Epoch 6/100
12/12 [========================

KeyboardInterrupt: 